1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다
    - 토큰 수 초과로 답변을 생성하지 못할 수 있음
    - 문서가 길면(인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 => 벡터 데이터베이스에 저장
4. 질문이 있을 때, 백터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [1]:
%pip install --upgrade --quiet docx2txt langchain-community

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("./tax.docx")
document = loader.load()

In [4]:
len(document)

1

In [5]:
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 하나의 청크가 가질 수 있는 크기
    chunk_overlap=200, # 청크를 쪼갤때 겹치는 범위 => 유사도 검색을 진행할 때 성공 확률을 높이기 위해
)
loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

In [9]:
len(document_list)

181

In [13]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [14]:
%pip install langchain-chroma

  Using cached uvicorn-0.35.0-py3-none-any.whl.metadata (6.5 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached bcrypt-4.3.0-cp39-abi3-win_amd64.whl.metadata (10 kB)
  Using cached kubernetes-33.1.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached rich-14.1.0-py3-none-any.whl.metadata (18 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached pyproject_hooks-1.2.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached tomli-2.2.1-py3-none-any.whl.metadata (10 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 

In [15]:
from langchain_chroma import Chroma

database = Chroma.from_documents(
    documents=document_list,
    embedding = embedding
)

In [17]:
query='연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrived_docs = database.similarity_search(query) # 유사도 검사

In [18]:
retrived_docs

[Document(id='1e903a0b-489d-40f3-88b8-611cca1b2f51', metadata={'source': './tax.docx'}, page_content='[전문개정 2009. 12. 31.]\n\n\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외

In [19]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [20]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrived_docs}

Question: {query}
"""

In [21]:
ai_message = llm.invoke(prompt)

In [22]:
ai_message.content

'연봉 5천만원인 직장인의 소득세를 계산하려면 몇 가지 정보를 추가로 확인해야 합니다. 소득세는 종합소득금액에서 공제 가능한 항목들을 차감한 후 과세표준 구간에 따른 세율을 적용하여 계산합니다. \n\n1. **종합소득공제**: 기본적으로 근로소득공제, 인적공제 등 기타 가능한 공제를 적용해야 합니다. 근로소득공제는 소득 구간에 따라 비율이 다르게 적용됩니다. \n\n2. **세율**: 현재 한국의 소득세율은 과세표준에 따라 다르게 적용됩니다. 예를 들어, 과세표준이\n   - 1,200만원 이하이면 6%,\n   - 1,200만원 초과 4,600만원 이하이면 15%,\n   - 4,600만원 초과 8,800만원 이하이면 24%\n   입니다.\n\n3. **지방소득세**: 소득세의 10%에 해당하는 금액이 추가로 지방소득세로 부과됩니다.\n\n직장인이 5천만원의 소득을 올린다면, 큰 틀에서 다음과 같이 계산할 수 있습니다:\n\n- **근로소득공제 및 기타 공제 적용 후 과세표준 계산**\n- **해당 과세표준에 대한 소득세율 적용**\n- **지방소득세 계산 및 합산**\n\n이러한 계산을 하기 위한 구체적인 공제 사항과 과세표준에 대한 정보는 개인별로 다를 수 있으므로, 정확한 소득세 액수를 계산하기 위해서는 보다 구체적인 정보가 필요합니다. 만약 기본적인 예시와 실제 세액 예시를 원하시면, 관련된 구체적인 정보를 제공해주시면 감사하겠습니다.'

In [23]:
from langchain_chroma import Chroma

database = Chroma.from_documents(
    documents=document_list,
    embedding = embedding,
    collection_name="chroma-tax",
    persist_directory="./chroma"
)

In [24]:
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.


In [25]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [26]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [34]:
from langchain.chains import RetrievalQA

database = Chroma(collection_name="chroma-tax", persist_directory="./chorma", embedding_function=embedding)

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=database.as_retriever(),
    chain_type_kwargs={
        "prompt": prompt
    }
)

In [35]:
ai_message = qa_chain({
    "query": query
})

In [36]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '소득세는 연봉 외에도 각종 공제와 세율에 따라 다르게 계산됩니다. 기본적으로 연봉 5천만원에 대한 소득세는 약간의 변동이 있을 수 있기에 구체적인 세율표나 세무 당국의 계산기를 참조하는 것이 좋습니다. 정확한 소득세 금액을 알고 싶다면, 세무서를 통해 자세한 정보를 확인하세요.'}